## Análise comparativa sem outliers

## 1. Validação e remoção de outliers

In [1]:
# Importando bibliotecas.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import metrics

In [2]:
# ler excel do repositório remoto.
df = pd.read_csv("https://raw.githubusercontent.com/atlantico-academy/datasets/refs/heads/main/diamonds.csv")

# Exibir as primeiras linhas do DataFrame.
print(df)

       carat        cut color clarity  depth  table  price     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72       Good     D     SI1   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    Premium     H     SI2   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]


In [3]:
## Contagem de valores ausentes.
print("\nValores ausentes em cada coluna:")
display(df.isnull().sum())
print("\n")

# Exibe os tipos de cada coluna.
print("Tipos de dados de cada coluna:")
print(df.dtypes)

# Filtra as linhas duplicadas e conta a quantidade.
duplicate = df[df.duplicated()]
num_duplicate = duplicate.shape[0]

# Exibe as linhas duplicadas e a contagem.
print(f"\nNúmero total de linhas duplicadas: {num_duplicate}")

# Remove as linhas duplicadas do dataset
df.drop_duplicates().reset_index(drop = True)


Valores ausentes em cada coluna:


carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64



Tipos de dados de cada coluna:
carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object

Número total de linhas duplicadas: 146


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53789,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53790,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53791,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53792,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [4]:
# Mudando nome de colunas.
df.rename(columns = {
            "cut":"corte", 
            "color":"cor", 
            "clarity":"claridade", 
            "depth": "profundidade", 
            "table":"mesa", 
            "price":"preço", 
            "x":"comprimento", 
            "y":"largura", 
            "z":"altura"
            }, inplace = True)

# Visualização dos primeiros registros.
df.head()

,carat,corte,cor,claridade,profundidade,mesa,preço,comprimento,largura,altura
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [5]:
# Calcular Q1, Q3 e IQR, especificando numeric_only=True
Q1 = df.quantile(0.25, numeric_only=True)
Q3 = df.quantile(0.75, numeric_only=True)
IQR = Q3 - Q1

# Definir limites inferior e superior
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Converter lower_bound e upper_bound para DataFrames para alinhar com df
lower_bound = pd.DataFrame([lower_bound] * len(df), columns=df.columns)
upper_bound = pd.DataFrame([upper_bound] * len(df), columns=df.columns)

# Remover outliers
df_no_outliers = df[~((df < lower_bound) | (df > upper_bound)).any(axis=1)]

# Contagem de outliers removidos
num_outliers_removed = len(df) - len(df_no_outliers)

print("DataFrame original com outliers:")
print(df.describe())

print("\nDataFrame sem outliers:")
print(df_no_outliers.describe())

# Imprimir o número de outliers removidos
print(f"\nNúmero de registros: {len(df)}")
print(f"Número de outliers removidos: {num_outliers_removed}")

DataFrame original com outliers:
              carat  profundidade          mesa         preço   comprimento  \
count  53940.000000  53940.000000  53940.000000  53940.000000  53940.000000   
mean       0.797940     61.749405     57.457184   3932.799722      5.731157   
std        0.474011      1.432621      2.234491   3989.439738      1.121761   
min        0.200000     43.000000     43.000000    326.000000      0.000000   
25%        0.400000     61.000000     56.000000    950.000000      4.710000   
50%        0.700000     61.800000     57.000000   2401.000000      5.700000   
75%        1.040000     62.500000     59.000000   5324.250000      6.540000   
max        5.010000     79.000000     95.000000  18823.000000     10.740000   

            largura        altura  
count  53940.000000  53940.000000  
mean       5.734526      3.538734  
std        1.142135      0.705699  
min        0.000000      0.000000  
25%        4.720000      2.910000  
50%        5.710000      3.530000  
75%

In [6]:
# Contar quantos registros têm valor 0 em 'comprimento', 'largura' e 'altura'
contagem_zero = {
    'comprimento': (df['comprimento'] == 0).sum(),
    'largura': (df['largura'] == 0).sum(),
    'altura': (df['altura'] == 0).sum()
}

# Remover linhas onde qualquer coluna de comprimento, altura ou largura é igual a 0
df_clean = df[(df['comprimento'] != 0) & (df['altura'] != 0) & (df['largura'] != 0)]

# Exibir o DataFrame resultante
print(df_clean)
print(contagem_zero)

       carat      corte cor claridade  profundidade  mesa  preço  comprimento  \
0       0.23      Ideal   E       SI2          61.5  55.0    326         3.95   
1       0.21    Premium   E       SI1          59.8  61.0    326         3.89   
2       0.23       Good   E       VS1          56.9  65.0    327         4.05   
3       0.29    Premium   I       VS2          62.4  58.0    334         4.20   
4       0.31       Good   J       SI2          63.3  58.0    335         4.34   
...      ...        ...  ..       ...           ...   ...    ...          ...   
53935   0.72      Ideal   D       SI1          60.8  57.0   2757         5.75   
53936   0.72       Good   D       SI1          63.1  55.0   2757         5.69   
53937   0.70  Very Good   D       SI1          62.8  60.0   2757         5.66   
53938   0.86    Premium   H       SI2          61.0  58.0   2757         6.15   
53939   0.75      Ideal   D       SI2          62.2  55.0   2757         5.83   

       largura  altura  
0 

In [7]:
# Definindo as ordens para as variáveis ordinais
ordem_corte = ['Ideal', 'Premium', 'Very Good', 'Good', 'Fair']
ordem_cor = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
ordem_claridade = ['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1']

# Instanciando o OrdinalEncoder para as colunas categóricas
encoder_corte = OrdinalEncoder(categories=[ordem_corte])
encoder_cor = OrdinalEncoder(categories=[ordem_cor])
encoder_claridade = OrdinalEncoder(categories=[ordem_claridade])

# Aplicando a codificação ordinal
df['corte'] = encoder_corte.fit_transform(df[['corte']])
df['cor'] = encoder_cor.fit_transform(df[['cor']])
df['claridade'] = encoder_claridade.fit_transform(df[['claridade']])

# Exibindo df
print(df)

       carat  corte  cor  claridade  profundidade  mesa  preço  comprimento  \
0       0.23    0.0  1.0        6.0          61.5  55.0    326         3.95   
1       0.21    1.0  1.0        5.0          59.8  61.0    326         3.89   
2       0.23    3.0  1.0        3.0          56.9  65.0    327         4.05   
3       0.29    1.0  5.0        4.0          62.4  58.0    334         4.20   
4       0.31    3.0  6.0        6.0          63.3  58.0    335         4.34   
...      ...    ...  ...        ...           ...   ...    ...          ...   
53935   0.72    0.0  0.0        5.0          60.8  57.0   2757         5.75   
53936   0.72    3.0  0.0        5.0          63.1  55.0   2757         5.69   
53937   0.70    2.0  0.0        5.0          62.8  60.0   2757         5.66   
53938   0.86    1.0  4.0        6.0          61.0  58.0   2757         6.15   
53939   0.75    0.0  0.0        6.0          62.2  55.0   2757         5.83   

       largura  altura  
0         3.98    2.43  
1

In [ ]:
# Separar as variáveis independentes (X) e dependentes (y)
X = df.drop('preço', axis=1)  # As variáveis independentes
y = df['preço']  # A variável dependente (preço)

# Divisão do dataset em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Colunas categóricas
colunas_categoricas = ['corte', 'cor', 'claridade']

# Colunas numéricas
colunas_numericas = X.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Definindo o pipeline para o modelo de regressão
def criar_pipeline(modelo):
    """
    Cria um pipeline que inclui transformação de dados (OneHotEncoder e StandardScaler) e treinamento do modelo.
    """
    # Transformador para pré-processar as colunas categóricas e numéricas
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), colunas_numericas),  # Escalonar as variáveis numéricas
            ('cat', OneHotEncoder(), colunas_categoricas)  # OneHotEncoder para as variáveis categóricas
        ])
    
    # Criar o pipeline com o pré-processamento e o modelo
    return Pipeline([
        ('preprocessor', preprocessor),  # Pré-processamento
        ('modelo', modelo)  # Modelo de regressão
    ])

# Modelos a serem utilizados
modelos = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(random_state=42)
}

# Avaliação dos modelos
resultados = {}

for nome, modelo in modelos.items():
    # Criar pipeline para o modelo
    pipeline = criar_pipeline(modelo)
    
    # Treinar o modelo
    pipeline.fit(X_train, y_train)
    
    # Previsões
    y_pred = pipeline.predict(X_test)
    
    # Calculando as métricas
    mse = np.sqrt(mean_squared_error(y_test, y_pred))
    rmse = mse**0.5  # Raiz do erro quadrático médio
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Guardando os resultados
    resultados[nome] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2
    }

# Exibindo os resultados
resultados_df = pd.DataFrame(resultados).T
print(resultados_df)

# Reorganizando a ordem das métricas
resultados_df_sorted = resultados_df[['MSE', 'MAE', 'RMSE', 'R²']]

# Gráfico para comparar os resultados
resultados_df.plot(kind='barh', figsize=(10, 6))
plt.title("Desempenho dos Modelos de Regressão")
plt.xlabel("Valor das Métricas")
plt.tight_layout()
plt.show()